In [1]:
from pprint import pprint
from shapely.geometry import mapping
import geopandas as gpd, rasterio, os, numpy as np, shapely, random
from PIL import Image
from rasterio.mask import mask
from rasterio.plot import reshape_as_image
from rasterio import features

orthomosaic_path = "D:/FCAT/FCAT2APPK.tif"
output_dir = "D:/FCAT/background_crops" # folder where any outputs will be dumped
input_csv = "D:/FCAT/Orthomosaic_1_training_data_v3.csv" # csv of points representing focal objects, here palm trees
class_colname = 'Especie' # column where the object "class" is located in the input_csv
object_size = 12 # estimated size of our focal object, here a palm tree, given in meters

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [2]:
### This section converts the raster footprint into a polygon called "all_polygons"

dataset = rasterio.open(orthomosaic_path)
band1 = dataset.read(1)
mask = band1 != 255

all_polygons = []
# for shape, value in features.shapes(mask.astype(np.int16), mask=(mask >0), transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0)):
for shape, value in features.shapes(mask.astype(np.int16), mask=(mask >0), transform=dataset.transform):
    all_polygons.append(shapely.geometry.shape(shape))

all_polygons = shapely.geometry.MultiPolygon(all_polygons)
if not all_polygons.is_valid:
    all_polygons = all_polygons.buffer(0)
    # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
    # need to keep it a Multi throughout
    if all_polygons.type == 'Polygon':
        all_polygons = shapely.geometry.MultiPolygon([all_polygons])

In [3]:
### This section converts the points from the CSV file to boxes
### not necessary if your annotations are already in boxes

# reading in the orthomosaic, for spatial reference (CRS)
# and the point file, as a csv with X and Y coordinate columns for lat/lon

dataset = rasterio.open(orthomosaic_path)
pts = gpd.read_file(input_csv) # read in the CSV file to a geodataframe
gdf=gpd.points_from_xy(pts.POINT_X, pts.POINT_Y)

# we apply a buffer around each point
buffer_dist = object_size # distance from center, in meters.
#Buffer will be 2x object size, since we're generating background points
buffer_dist = buffer_dist/111300 # converted to lat/lon degrees

box_list = []
for pt in gdf:
    box= pt.buffer(buffer_dist).envelope
    box_list.append(box)

# then we assemble a new geodataframe with these boxes

d = {class_colname: pts[class_colname], 'geometry': box_list}
boxes = gpd.GeoDataFrame(d, crs=dataset.crs)

# we now have a geodataframe of species-labeled boxes extending 'buffer_dist' around each point
print(boxes[0:5])

             Especie                                           geometry
0    Attalea colenda  POLYGON ((-79.66503 0.36755, -79.66481 0.36755...
1  Oenocarpus bataua  POLYGON ((-79.66449 0.36788, -79.66428 0.36788...
2  Oenocarpus bataua  POLYGON ((-79.66544 0.36798, -79.66523 0.36798...
3  Oenocarpus bataua  POLYGON ((-79.66852 0.36822, -79.66831 0.36822...
4  Oenocarpus bataua  POLYGON ((-79.66944 0.36872, -79.66923 0.36872...


In [4]:
### this step takes a long time
gdf1 = gpd.GeoDataFrame(index=[0], crs=dataset.crs, geometry=[all_polygons])
gdf2 = gdf1.overlay(boxes, how='difference')

### If you want to view this file:
# gdf2.to_file(output_dir + '/background.shp')

In [28]:
### this step also takes a long time
def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = shapely.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
            if len(points) % 100 == 0:
                print(f'{len(points)} of {number} points generated so far')
    print('done.')
    return points

background_pts = generate_random(3128, gdf2['geometry'][0])

100 of 3128 points generated so far
200 of 3128 points generated so far
300 of 3128 points generated so far
400 of 3128 points generated so far
500 of 3128 points generated so far
600 of 3128 points generated so far
700 of 3128 points generated so far
800 of 3128 points generated so far
900 of 3128 points generated so far
1000 of 3128 points generated so far
1100 of 3128 points generated so far
1200 of 3128 points generated so far
1300 of 3128 points generated so far
1400 of 3128 points generated so far
1500 of 3128 points generated so far
1600 of 3128 points generated so far
1700 of 3128 points generated so far
1800 of 3128 points generated so far
1900 of 3128 points generated so far
2000 of 3128 points generated so far
2100 of 3128 points generated so far
2200 of 3128 points generated so far
2300 of 3128 points generated so far
2400 of 3128 points generated so far
2500 of 3128 points generated so far
2600 of 3128 points generated so far
2700 of 3128 points generated so far
2800 of 31

In [76]:
### If you want to export it as a CSV

import csv
coords = [[x.coords[0][0], x.coords[0][1]] for x in background_pts]
with open(output_dir + "/background_coords.csv", 'w') as f:
    write = csv.writer(f, lineterminator='\r')
    write.writerows(coords)

In [29]:
### If you want to export it as a shapefile

from shapely.geometry import mapping, MultiPoint
import fiona

background_mpts = shapely.MultiPoint(background_pts)

# Define a polygon feature geometry with one attribute
schema = {
    'geometry': 'MultiPoint',
    'properties': {'id': 'int'},
}

# Write a new Shapefile
with fiona.open(output_dir + '/background_points.shp', 'w', 'ESRI Shapefile', schema) as c:
    ## If there are multiple geometries, put the "for" loop here
    c.write({
        'geometry': mapping(background_mpts),
        'properties': {'id': 111},
    })